In [ ]:
import ROOT as r
from ROOT import TH1D, TFile, TChain, TCanvas, TBranchElement, TLorentzVector

luminosity_ifb = 10

def createDirs(path):
    """
    Function to create any directories needed to store the output of a function.

    Args:
        path (str): the path of directories you want to exist

    Returns:
        void
    """
    base = path.split("/")[0]
    dirs = path.split("/")[:]
    tmp_dir = ""
    for folder in dirs:
        tmp_dir = tmp_dir + "/" + folder
        if not os.path.isdir(base+"/"+tmp_dir):
            try:
                os.mkdir(base+"/"+tmp_dir)
                print("creating: ", base+"/"+tmp_dir)
            except OSError as error:
                print(error)

# Define out input and output paths, make sure the output path exists.
hist_path = "./histograms/GamGam_root/"
plot_path = "./plots/GamGam_root/"
createDirs(output_path)

In [ ]:
def plotStack(hists):
    c = TCanvas()
    stack = TStack()
    for hist in hists:
        


In [ ]:
def plotWithFit(datahist, signalhist):
    c = TCanvas()
    data = datahist